# Bidirectional LSTM

In [1]:
from matplotlib import pyplot as plt

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import numpy as np

import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. Load data

In [2]:
def load_json(path):
    '''
    Loads the JSON file of the Squad dataset.
    Returns the json object of the dataset.
    '''
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    print("Length of data: ", len(data['data']))
    print("Data Keys: ", data['data'][0].keys())
    print("Title: ", data['data'][0]['title'])
    
    return data

def parse_data(data:dict)->list:
    '''
    Parses the JSON file of Squad dataset by looping through the
    keys and values and returns a list of dictionaries with
    context, query and label triplets being the keys of each dict.
    '''
    data = data['data']
    qa_list = []

    for paragraphs in data:

        for para in paragraphs['paragraphs']:
            context = para['context']

            for qa in para['qas']:
                
                id = qa['id']
                question = qa['question']
                
                for ans in qa['answers']:
                    answer = ans['text']
                    ans_start = ans['answer_start']
                    ans_end = ans_start + len(answer)
                    
                    qa_dict = {}
                    qa_dict['id'] = id
                    qa_dict['context'] = context
                    qa_dict['question'] = question
                    qa_dict['label'] = [ans_start, ans_end]

                    qa_dict['answer'] = answer
                    qa_list.append(qa_dict)    

    
    return qa_list


In [3]:
import json

train_data = load_json('/kaggle/input/new-squaddataset/train-v2.0.json')
valid_data = load_json('/kaggle/input/new-squaddataset/dev-v2.0.json')

# parse the json structure to return the data as a list of dictionaries

train_list = parse_data(train_data)
valid_list = parse_data(valid_data)
print('--------------------------')

print('Train list len: ',len(train_list))
print('Valid list len: ',len(valid_list))

# converting the lists into dataframes

train_ds = pd.DataFrame(train_list)
val_ds = pd.DataFrame(valid_list)

train_ds = train_ds.drop('id', axis=1)
train_ds = train_ds.drop('label', axis=1)
val_ds = val_ds.drop('id', axis=1)
val_ds = val_ds.drop('label', axis=1)
train_ds.columns = ['Paragraph', 'Question', 'Answer']
val_ds.columns = ['Paragraph', 'Question', 'Answer']

Length of data:  442
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  University_of_Notre_Dame
Length of data:  48
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Super_Bowl_50
--------------------------
Train list len:  87599
Valid list len:  34726


In [4]:
train_ds.head()

,Paragraph,Question,Answer
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary


In [5]:
val_ds.head()

,Paragraph,Question,Answer
0,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,Denver Broncos
1,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,Denver Broncos
2,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,Denver Broncos
3,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,Carolina Panthers
4,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,Carolina Panthers


# 2. Pre-processing

## 2.1. Tokenization

In [6]:
%time train_ds['Paragraph'] = train_ds['Paragraph'].apply(nltk.word_tokenize)
%time train_ds['Question'] = train_ds['Question'].apply(nltk.word_tokenize)
%time train_ds['Answer'] = train_ds['Answer'].apply(nltk.word_tokenize)
%time val_ds['Paragraph'] = val_ds['Paragraph'].apply(nltk.word_tokenize)
%time val_ds['Question'] = val_ds['Question'].apply(nltk.word_tokenize)
%time val_ds['Answer'] = val_ds['Answer'].apply(nltk.word_tokenize)

CPU times: user 36.8 s, sys: 360 ms, total: 37.1 s
Wall time: 37.2 s
CPU times: user 4.85 s, sys: 38.5 ms, total: 4.89 s
Wall time: 4.89 s
CPU times: user 3.5 s, sys: 20.7 ms, total: 3.52 s
Wall time: 3.52 s
CPU times: user 16.5 ms, sys: 0 ns, total: 16.5 ms
Wall time: 16.5 ms
CPU times: user 2.43 ms, sys: 0 ns, total: 2.43 ms
Wall time: 2.44 ms
CPU times: user 1.94 ms, sys: 0 ns, total: 1.94 ms
Wall time: 1.95 ms


In [7]:
train_ds.head()

,Paragraph,Question,Answer
0,"[Architecturally, ,, the, school, has, a, Cath...","[To, whom, did, the, Virgin, Mary, allegedly, ...","[Saint, Bernadette, Soubirous]"
1,"[Architecturally, ,, the, school, has, a, Cath...","[What, is, in, front, of, the, Notre, Dame, Ma...","[a, copper, statue, of, Christ]"
2,"[Architecturally, ,, the, school, has, a, Cath...","[The, Basilica, of, the, Sacred, heart, at, No...","[the, Main, Building]"
3,"[Architecturally, ,, the, school, has, a, Cath...","[What, is, the, Grotto, at, Notre, Dame, ?]","[a, Marian, place, of, prayer, and, reflection]"
4,"[Architecturally, ,, the, school, has, a, Cath...","[What, sits, on, top, of, the, Main, Building,...","[a, golden, statue, of, the, Virgin, Mary]"


In [8]:
val_ds.head()

,Paragraph,Question,Answer
0,"[Super, Bowl, 50, was, an, American, football,...","[Which, NFL, team, represented, the, AFC, at, ...","[Denver, Broncos]"
1,"[Super, Bowl, 50, was, an, American, football,...","[Which, NFL, team, represented, the, AFC, at, ...","[Denver, Broncos]"
2,"[Super, Bowl, 50, was, an, American, football,...","[Which, NFL, team, represented, the, AFC, at, ...","[Denver, Broncos]"
3,"[Super, Bowl, 50, was, an, American, football,...","[Which, NFL, team, represented, the, NFC, at, ...","[Carolina, Panthers]"
4,"[Super, Bowl, 50, was, an, American, football,...","[Which, NFL, team, represented, the, NFC, at, ...","[Carolina, Panthers]"


In [9]:
# Compute maximum length statistics for paragraph and question
paragraph_length = max(train_ds['Paragraph'].map(len).max(), val_ds['Paragraph'].map(len).max())
question_length = max(train_ds['Question'].map(len).max(), val_ds['Question'].map(len).max())
print('Max paragraph length:', paragraph_length)
print('Max question length:', question_length)

Max paragraph length: 618
Max question length: 60


## 2.2. Encode answers

In [10]:
num_not_found = 0
not_found = []

# Map answer tokens to one-hot encodings of start and end positions of the answer span extracted from the paragraph
def encode_answer(paragraph_tokens, answer_tokens):
    global num_not_found, not_found
    answer_ptr = 0
    start_pos = None
    end_pos = None
    for i, paragraph_token in enumerate(paragraph_tokens):
        if paragraph_token == answer_tokens[answer_ptr]:
            if start_pos == None:
                start_pos = i
            answer_ptr += 1
            if answer_ptr == len(answer_tokens):
                end_pos = i
                break
        elif start_pos != None:
            start_pos = None
            end_pos = None

    start = [0] * paragraph_length
    end = [0] * paragraph_length

    if start_pos == None or end_pos == None:
        num_not_found += 1
        not_found.append([paragraph_tokens, answer_tokens])
    else:
        start[start_pos] = 1
        end[end_pos] = 1
    return [start, end]

train_ds['Answer'] = train_ds.apply(lambda row: encode_answer(row['Paragraph'], row['Answer']), axis=1)
val_ds['Answer'] = val_ds.apply(lambda row: encode_answer(row['Paragraph'], row['Answer']), axis=1)


In [11]:
train_ds.head()

,Paragraph,Question,Answer
0,"[Architecturally, ,, the, school, has, a, Cath...","[To, whom, did, the, Virgin, Mary, allegedly, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"[Architecturally, ,, the, school, has, a, Cath...","[What, is, in, front, of, the, Notre, Dame, Ma...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[Architecturally, ,, the, school, has, a, Cath...","[The, Basilica, of, the, Sacred, heart, at, No...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,..."
3,"[Architecturally, ,, the, school, has, a, Cath...","[What, is, the, Grotto, at, Notre, Dame, ?]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"[Architecturally, ,, the, school, has, a, Cath...","[What, sits, on, top, of, the, Main, Building,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


# 3. Embeddings - GloVe

In [12]:
embedding_file = 'glove.6B.50d.txt'
embedding_size = 50

# # Pre-computed unknown vector (by taking average of all word vectors)
# # Reference: https://stackoverflow.com/questions/49239941/what-is-unk-in-the-pretrained-glove-vector-files-e-g-glove-6b-50d-txt
unknown_vector = np.array([-0.12920076, -0.28866628, -0.01224866, -0.05676644, -0.20210965, -0.08389011,
    0.33359843,  0.16045167,  0.03867431,  0.17833012,  0.04696583, -0.00285802,
    0.29099807,  0.04613704,  -0.20923874, -0.06613114, -0.06822549, 0.07665912,
    0.3134014,   0.17848536,  -0.1225775,  -0.09916984, -0.07495987, 0.06413227,
    0.14441176,  0.60894334,  0.17463093,  0.05335403,  -0.01273871, 0.03474107,
    -0.8123879,  -0.04688699, 0.20193407,  0.2031118,   -0.03935686, 0.06967544,
    -0.01553638, -0.03405238, -0.06528071, 0.12250231,  0.13991883, -0.17446303,
    -0.08011883, 0.0849521,   -0.01041659, -0.13705009, 0.20127155, 0.10069408,
    0.00653003,  0.01685157], np.float32)

In [13]:
import numpy as np
import io

def get_glove_dict():
    '''
    Parses the glove word vectors text file and returns a dictionary with the words as
    keys and their respective pretrained word vectors as values.
    '''
    encoding = 'utf-8'
    glove_dict = {}
    with open("/kaggle/input/glove6b50dtxt/glove.6B.50d.txt", "r") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            glove_dict[word] = vector

    return glove_dict


In [14]:
embeddings = get_glove_dict()

In [15]:
# Transform word tokens into word embeddings
def embed(tokens):
    vectors = []
    for token in tokens:
        token = token.lower()
        if token in embeddings:
            vectors.append(embeddings[token])
        else:
            vectors.append(unknown_vector)
    return np.asarray(vectors, np.float32)

train_ds['Paragraph'], train_ds['Question'] = train_ds['Paragraph'].map(embed), train_ds['Question'].map(embed)
val_ds['Paragraph'], val_ds['Question'] = val_ds['Paragraph'].map(embed), val_ds['Question'].map(embed)

In [16]:
train_ds.head()

,Paragraph,Question,Answer
0,"[[0.99697, 0.28959, -0.53768, -0.29199, 0.7795...","[[0.68047, -0.039263, 0.30186, -0.17792, 0.429...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"[[0.99697, 0.28959, -0.53768, -0.29199, 0.7795...","[[0.45323, 0.059811, -0.10577, -0.333, 0.72359...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[[0.99697, 0.28959, -0.53768, -0.29199, 0.7795...","[[0.418, 0.24968, -0.41242, 0.1217, 0.34527, -...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,..."
3,"[[0.99697, 0.28959, -0.53768, -0.29199, 0.7795...","[[0.45323, 0.059811, -0.10577, -0.333, 0.72359...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"[[0.99697, 0.28959, -0.53768, -0.29199, 0.7795...","[[0.45323, 0.059811, -0.10577, -0.333, 0.72359...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


# 4. Prepare training and validation data

In [17]:
# Pad paragraph and question embeddings
def pad_paragraph(vectors):
    remaining_length = paragraph_length - len(vectors)
    paddings = np.repeat([np.zeros(embedding_size)], remaining_length, axis=0)
    return np.concatenate((vectors, paddings), axis=0, dtype=np.float32)

def pad_question(vectors):
    remaining_length = question_length - len(vectors)
    paddings = np.repeat([np.zeros(embedding_size)], remaining_length, axis=0)
    return np.concatenate((vectors, paddings), axis=0, dtype=np.float32)

paragraph_train = train_ds['Paragraph'].map(pad_paragraph).to_list()
question_train = train_ds['Question'].map(pad_question).to_list()

paragraph_val = val_ds['Paragraph'].map(pad_paragraph).to_list()
question_val = val_ds['Question'].map(pad_question).to_list()

In [18]:
# Get start and end token positions
answer_train = train_ds['Answer'].to_list()
start_train = [ans[0] for ans in answer_train]
end_train = [ans[1] for ans in answer_train]

answer_val = val_ds['Answer'].to_list()
start_val = [ans[0] for ans in answer_val]
end_val = [ans[1] for ans in answer_val]

In [19]:
# Convert to constant tensor
paragraph_train, question_train = tf.constant(paragraph_train, np.float32), tf.constant(question_train, np.float32)
paragraph_val, question_val = tf.constant(paragraph_val, np.float32), tf.constant(question_val, np.float32)
start_train, end_train = tf.constant(start_train, np.float32), tf.constant(end_train, np.float32)
start_val, end_val = tf.constant(start_val, np.float32), tf.constant(end_val, np.float32)

# 5. Feature Extraction - Bidirectional LSTM encoder

In [20]:
# LSTM encoder layer for paragraph
paragraph_inputs = tf.keras.Input(shape=(paragraph_length, embedding_size))
paragraph_inputs = tf.keras.layers.Masking(mask_value=np.zeros(embedding_size))(paragraph_inputs)
paragraph_encoded = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_size, return_sequences=True))(paragraph_inputs)

# LSTM encoder layer for question
question_inputs = tf.keras.Input(shape=(question_length, embedding_size))
question_inputs = tf.keras.layers.Masking(mask_value=np.zeros(embedding_size))(question_inputs)
question_encoded = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_size, return_sequences=True))(question_inputs)

# 6. Paragraph-Question Interaction - Bidirectional attention + One-hop interaction

In [21]:
# Co-attention layer
scores = tf.matmul(paragraph_encoded, question_encoded, transpose_b=True)
question_weights = tf.nn.softmax(scores)
paragraph_weights = tf.nn.softmax(tf.transpose(scores, perm=[0,2,1]))
question_context = tf.matmul(paragraph_encoded, question_weights, transpose_a=True)
question_concat = tf.concat([tf.transpose(question_encoded, perm=[0,2,1]), question_context], axis=1)
paragraph_context = tf.transpose(tf.matmul(question_concat, paragraph_weights), perm=[0,2,1])

# 7. Span prediction - Unidirectional boundary model

In [22]:
# Answer pointer layer
boundary = tf.keras.layers.LSTM(embedding_size, return_sequences=True)(paragraph_context)
start_pos_logits = tf.keras.layers.Dense(1)(boundary)
boundary = tf.keras.layers.Concatenate()([boundary, start_pos_logits])
end_pos_logits = tf.keras.layers.Dense(1)(boundary)

start_pos_distribution = tf.keras.layers.Softmax()(tf.squeeze(start_pos_logits, axis=-1))
end_pos_distribution = tf.keras.layers.Softmax()(tf.squeeze(end_pos_logits, axis=-1))

# 8. Build, fit and evaluate model

In [3]:
def cross_entropy_loss(y_true, y_pred):
    start_true, end_true = y_true[0], y_true[1]
    start_pred, end_pred = y_pred[0], y_pred[1]

    start_loss = -tf.reduce_sum(start_true * tf.math.log(start_pred))
    end_loss = -tf.reduce_sum(end_true * tf.math.log(end_pred))

    total_loss = start_loss + end_loss
    return total_loss

def exact_match(y_true, y_pred):
    start_true, end_true = tf.math.argmax(y_true[0], output_type=tf.int32), tf.math.argmax(y_true[1], output_type=tf.int32)
    start_pred = tf.math.argmax(y_pred[0], output_type=tf.int32)
    end_pred = start_pred + tf.math.argmax(y_pred[1][start_pred:], output_type=tf.int32)

    if start_pred != start_true or end_pred != end_true:
        return float(0)
    else:
        return float(1)

def f1_score(y_true, y_pred):
    start_true, end_true = tf.math.argmax(y_true[0], output_type=tf.int32), tf.math.argmax(y_true[1], output_type=tf.int32)
    start_pred = tf.math.argmax(y_pred[0], output_type=tf.int32)
    end_pred = start_pred + tf.math.argmax(y_pred[1][start_pred:], output_type=tf.int32)

    start_min = tf.math.minimum(start_true, start_pred)
    end_max = tf.math.maximum(end_true, end_pred)

    true_pos = 0
    false_neg = 0
    false_pos = 0

    for pos in range(start_min, end_max + 1):
        in_true = start_true <= pos <= end_true
        in_pred = start_pred <= pos <= end_pred

        if in_true:
            if in_pred:
                true_pos += 1
            else:
                false_neg += 1
        else:
            if in_pred:
                false_pos += 1

    if true_pos == 0 and false_neg == 0 and false_pos == 0:
        return float(0)

    return float((2 * true_pos) / (2 * true_pos + false_pos + false_neg))

model = tf.keras.Model(inputs=[paragraph_inputs, question_inputs], outputs =[start_pos_distribution, end_pos_distribution])
model.compile(optimizer='adam', loss=cross_entropy_loss, metrics=[exact_match, f1_score])
model.summary()

NameError: name 'paragraph_inputs' is not defined

In [ ]:
epochs = 4
history = model.fit(
    [paragraph_train, question_train],
    [start_train, end_train],
    epochs=epochs,
    validation_data=([paragraph_val, question_val], [start_val, end_val])
)

Epoch 1/4
1250/1250 [==============================] - 758s 598ms/step - loss: 16.0278 - softmax_loss: 8.0878 - softmax_1_loss: 7.9400 - softmax_exact_match: 0.0056 - softmax_f1_score: 0.1669 - softmax_1_exact_match: 0.0088 - softmax_1_f1_score: 0.1827 - val_loss: 19.1832 - val_softmax_loss: 10.1884 - val_softmax_1_loss: 8.9949 - val_softmax_exact_match: 0.0000e+00 - val_softmax_f1_score: 0.0000e+00 - val_softmax_1_exact_match: 0.0000e+00 - val_softmax_1_f1_score: 0.0000e+00
Epoch 2/4
 627/1250 [==============>...............] - ETA: 6:05 - loss: 14.3237 - softmax_loss: 7.3206 - softmax_1_loss: 7.0032 - softmax_exact_match: 0.0175 - softmax_f1_score: 0.2046 - softmax_1_exact_match: 0.0128 - softmax_1_f1_score: 0.1940

In [4]:
em = history.history['softmax_1_exact_match']
f1 = history.history['softmax_1_f1_score']
loss = history.history['loss']

val_em = history.history['val_softmax_1_exact_match']
val_f1 = history.history['val_softmax_1_f1_score']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, em, label='Training EM')
plt.plot(epochs_range, val_em, label='Validation EM')
plt.legend(loc='lower right')
plt.title('Bidirectional LSTM EM')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, f1, label='Training F1')
plt.plot(epochs_range, val_f1, label='Validation F1')
plt.legend(loc='lower right')
plt.title('Bidirectional LSTM F1')
plt.show()

plt.subplot(2, 2, 3)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Bidirectional LSTM Loss')
plt.show()

NameError: name 'history' is not defined

In [ ]:
results = model.predict([paragraph_val, question_val])
predictions = []
for start_dist, end_dist in zip(results[0], results[1]):
    start_pred = np.argmax(start_dist)
    end_pred = start_pred + np.argmax(end_dist[start_pred:])
    predictions.append([start_pred, end_pred])
with open('model3_val.npy', 'wb') as f:
    np.save(f, np.array(predictions))

In [ ]:
results = model.predict([paragraph_train, question_train])
predictions = []
for start_dist, end_dist in zip(results[0], results[1]):
    start_pred = np.argmax(start_dist)
    end_pred = start_pred + np.argmax(end_dist[start_pred:])
    predictions.append([start_pred, end_pred])
with open('model3_train.npy', 'wb') as f:
    np.save(f, np.array(predictions))

In [ ]:
!ls

In [ ]:
# predictions

In [6]:
with open('/kaggle/working/model3_val.npy', 'rb') as f:
    model3_val = np.load(f)

with open('/kaggle/working/model3_train.npy', 'rb') as f:
    model3_train = np.load(f)

In [8]:
train_data_testing = load_json('/kaggle/input/new-squaddataset/train-v2.0.json')
valid_data_testing = load_json('/kaggle/input/new-squaddataset/dev-v2.0.json')

# parse the json structure to return the data as a list of dictionaries

train_list_testing = parse_data(train_data_testing)
valid_list_testing = parse_data(valid_data_testing)
print('--------------------------')

print('Train list len: ',len(train_list_testing))
print('Valid list len: ',len(valid_list_testing))

# converting the lists into dataframes

train_ds_testing = pd.DataFrame(train_list_testing)
val_ds_testing = pd.DataFrame(valid_list_testing)
# train_ds_testing.columns = ['Paragraph', 'Question', 'Answer']
# val_ds_testing.columns = ['Paragraph', 'Question', 'Answer']

Length of data:  442
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  University_of_Notre_Dame
Length of data:  48
Data Keys:  dict_keys(['title', 'paragraphs'])
Title:  Super_Bowl_50
--------------------------
Valid list len:  34726


In [9]:
val_ds_testing.head()

,id,context,question,label,answer
0,56be4db0acb8001400a502ec,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"[177, 191]",Denver Broncos
1,56be4db0acb8001400a502ec,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"[177, 191]",Denver Broncos
2,56be4db0acb8001400a502ec,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"[177, 191]",Denver Broncos
3,56be4db0acb8001400a502ed,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,"[249, 266]",Carolina Panthers
4,56be4db0acb8001400a502ed,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,"[249, 266]",Carolina Panthers


In [10]:
predictions = {}
for index, row in val_ds_testing.iterrows():
    id_val = row['id']
    paragraph = row['context']
    question = row['question']
    answer_start = row['label'][0]
    answer = row['answer']
    
    answer_end = answer_start + len(answer)
    if paragraph[answer_start-1:answer_end-1] == answer:
        answer_start -= 1
        answer_end -= 1
    elif paragraph[answer_start-2:answer_end-2] == answer:
        answer_start -= 2
        answer_end -= 2

    m3_start = model3_val[index][0]
    m3_end = model3_val[index][1]
    m3 = paragraph[m3_start:m3_end+1]
    
    predicted_answer = m3
    actual_answer = answer

    # Convert predicted and actual answers to sets of characters
    predicted_chars = set(predicted_answer)
    actual_chars = set(actual_answer)

    # Calculate common characters
    common_chars = predicted_chars.intersection(actual_chars)

    # Calculate precision, recall, and F1 score at character level
    precision = len(common_chars) / len(predicted_chars) if len(predicted_chars) > 0 else 0
    recall = len(common_chars) / len(actual_chars) if len(actual_chars) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    
    predictions[id_val] = [m3,f1_score]

prediction_validDataset_BiLSTM = 'prediction_validDataset_BiLSTM.txt'
with open(prediction_validDataset_BiLSTM, 'w', encoding='utf-8') as file:
    json.dump(predictions, file, ensure_ascii=False)

In [ ]:
predictions = {}
for index, row in train_ds_testing.iterrows():
    id_val = row['id']
    paragraph = row['context']
    question = row['question']
    answer_start = row['label'][0]
    answer = row['answer']
    
    answer_end = answer_start + len(answer)
    if paragraph[answer_start-1:answer_end-1] == answer:
        answer_start -= 1
        answer_end -= 1
    elif paragraph[answer_start-2:answer_end-2] == answer:
        answer_start -= 2
        answer_end -= 2

    m3_start = model3_train[index][0]
    m3_end = model3_train[index][1]
    m3 = paragraph[m3_start:m3_end+1]
    
    predicted_answer = m3
    actual_answer = answer

    # Convert predicted and actual answers to sets of characters
    predicted_chars = set(predicted_answer)
    actual_chars = set(actual_answer)

    # Calculate common characters
    common_chars = predicted_chars.intersection(actual_chars)

    # Calculate precision, recall, and F1 score at character level
    precision = len(common_chars) / len(predicted_chars) if len(predicted_chars) > 0 else 0
    recall = len(common_chars) / len(actual_chars) if len(actual_chars) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    
    predictions[id_val] = [m3,f1_score]

prediction_trainDataset_BiLSTM = 'prediction_trainDataset_BiLSTM.txt'
with open(prediction_trainDataset_BiLSTM, 'w', encoding='utf-8') as file:
    json.dump(predictions, file, ensure_ascii=False)

In [ ]:
# predictions

In [ ]:
total_sum = 0
num_elements = 0

for key in predictions.keys():
    total_sum += predictions[key][1]
    num_elements += 1

if num_elements > 0:
    average = total_sum / num_elements
else:
    average = 0

# **F1 Score**

In [12]:
print(average)

NameError: name 'average' is not defined